***
## Класс формы для создания нового пользователя

В админ-зоне Django уже есть работающая форма, через которую администратор проекта может создать нового пользователя. Этой формой управляет класс `UserCreationForm` из модуля `django.contrib.auth.forms`.

![alt text](https://pictures.s3.yandex.net/resources/212_1682601901.png)

Это класс формы, наследник класса `ModelForm`; «под капотом» он связан со встроенной моделью пользователя.

В классе `UserCreationForm` есть дополнительные методы, необходимые для создания нового объекта пользователя, например валидация вводимых данных и проверка надёжности пароля.

Возьмём готовый класс формы регистрации пользователей и на его основе создадим систему регистрации, доступную пользователям.

***
## Маршрут и view-класс для страницы регистрации

Маршрут для страницы регистрации возьмём *auth/registration/*. Можно выбрать любой другой, но в проекте все маршруты, связанные с аутентификацией, описаны через префикс *auth/* — так что последуем этому принципу.

Регистрация нового пользователя — это всего лишь создание нового объекта в модели. Для создания объектов есть стандартный класс CreateView — его и применим для создания страницы регистрации. 

Чтобы он заработал — в него надо передать 

* `template_name` — название HTML-шаблона,

* `form_class` — класс формы,

* `success_url` — адрес для переадресации в случае успешного создания пользователя.

Чтобы подключить CBV `CreateView`, есть знакомый путь: 

* в файле *views.py* создать собственный класс, унаследовав его от `CreateView`,

* передать в него имя шаблона, формы и адрес редиректа,

* вызвать этот класс в маршрутизаторе с помощью метода `as_view()`.

Но есть другой подход: вызвать класс `CreateView` можно прямо в файле *urls.py*, а все нужные атрибуты передать аргументами в метод `as_view()`:

```py
# acme_project/urls.py

# Добавьте новые строчки с импортами классов.
from django.contrib.auth.forms import UserCreationForm
from django.views.generic.edit import CreateView
...
# К импортам из django.urls добавьте импорт функции reverse_lazy
from django.urls import include, path, reverse_lazy

urlpatterns = [
    ...
    path(
        'auth/registration/', 
        CreateView.as_view(
            template_name='registration/registration_form.html',
            form_class=UserCreationForm,
            success_url=reverse_lazy('pages:homepage'),
        ),
        name='registration',
    ),
    ...
] 
```

Применим этот подход: в проекте нет отдельного приложения для работы с пользователями, и разместить CBV некуда; создавать приложение ради единственного view-класса не хочется. 

***
## Шаблон для страницы регистрации

Теперь шаблон: он будет точно таким же, как любой из имеющихся в директории *registration/* шаблонов с формой — *password_reset_form.html* или *password_change_form.html*. 

Единственным отличием будет текстовый заголовок и надпись на кнопке.

Все шаблоны, связанные с аутентификацией, собраны в директории *registration/* — шаблон страницы регистрации сохраним там же.

Создайте шаблон *registration/registration_form.html* и перенесите в него код из листинга:

```html
{% extends "base.html" %}
{% load django_bootstrap5 %}

{% block content %}
  <div class="card col-4 m-3">
    <div class="card-header">
      Регистрация пользователя
    </div>
    <div class="card-body">
      <form method="post">
        {% csrf_token %}
        {% bootstrap_form form %}
        {% bootstrap_button button_type="submit" content="Зарегистрироваться" %}
      </form>
    </div>
  </div>
{% endblock %}
```

Готово! Можно проверить работу формы регистрации. Откройте страницу http://127.0.0.1:8000/auth/registration/:

***
## Регистрация пользователя при переопределении модели

Класс формы `UserCreationForm` рассчитан на работу со встроенной моделью пользователя; если же модель переопределена — придётся переопределить и класс формы.

Для этого нужно будет создать собственный класс — наследник класса `UserCreationForm`; в нём можно будет указать новую модель пользователя. И указывать новую модель лучше всего через `get_user_model()` — как и в любом коде, где идёт обращение к модели пользователя.

Если в проекте переопределена модель `User` — начинает накапливаться код, связанный с пользователями: описана кастомная модель, надо создать форму, настроить пути... Для сохранения порядка в проекте имеет смысл создать приложение users и хранить в нём всё, что связано с управлением пользователями:

* admin.py — файл с настройками админки для модели пользователя;

* forms.py — наследник формы `UserCreationForm`;

* models.py — кастомная модель пользователя;

* urls.py — подключаемые URL из `django.contrib.auth`, а также путь для регистрации;

* views.py — CBV для страницы регистрации пользователя.

В файле *forms.py* создаётся собственный класс формы, унаследованный от `UserCreationForm`. Единственное, зачем нужен этот класс — переопределить модель, с которой работает форма. Для этого в классе `CustomUserCreationForm` переопределяется атрибут `model` подкласса `Meta`: в атрибуте `model` указывается модель для формы.

```py
# users/forms.py
from django.contrib.auth.forms import UserCreationForm
from django.contrib.auth import get_user_model

# Получаем модель пользователя:
User = get_user_model()


class CustomUserCreationForm(UserCreationForm):

    # Наследуем класс Meta от соответствующего класса родительской формы.
    # Так этот класс будет не перезаписан, а расширен.
    class Meta(UserCreationForm.Meta):
        model = User 
```

Если класс `Meta` не наследовать от родительского `UserCreationForm.Meta`, то он будет переопределён целиком, а нам этого не нужно: необходимо изменить лишь модель, с которой связана форма, а все остальные атрибуты этого класса должны унаследоваться от родительского класса.

В классе `Meta` можно переопределить и атрибут `fields` — список полей, которые будут выведены в HTML-форму. Например, если пользователь сразу при регистрации должен заполнить поле с биографией (поле модели `bio`), то это поле должно быть указано в `fields`:

```py
...

class CustomUserCreationForm(UserCreationForm):

    class Meta(UserCreationForm.Meta):
        model = User
        fields = ('username', 'bio') 
```

Поля для пароля в атрибуте `fields` перечислять не надо, они выводятся в форму автоматически — таковы свойства родительского класса.

***
## Обновление навигации сайта

Страницы регистрации и смены пароля доступны и работают, но, чтобы на них попасть, приходится руками вводить адрес в строке браузера. Это неудобно, а новый пользователь и вовсе не сможет угадать нужные адреса. 

Добавим ссылки на эти страницы в навигацию.

```html
<!-- templates/includes/header.html -->
<header>
  <nav class="navbar shadow-sm">
    <div class="container">
      {% with request.resolver_match.view_name as view_name %}
        <ul class="nav nav-pills">
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'pages:homepage' %} active {% endif %}"
              href="{% url 'pages:homepage' %}">
              Главная
            </a>
          </li>
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'birthday:create' %} active {% endif %}"
              href="{% url 'birthday:create' %}">
              Калькулятор дней рождения
            </a>
          </li>
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'birthday:list' %} active {% endif %}" href="{% url 'birthday:list' %}">
              Cписок дней рождения
            </a>
          </li>
          {% if user.is_authenticated %}
            <span class="navbar-text">Пользователь: <b>{{ user.username }}</b></span>
            <!-- Новая кнопка -->
            <li class="nav-item">
              <a class="nav-link{% if view_name == 'password_change' %} active {% endif %}"
                href="{% url 'password_change' %}">Изменить пароль</a>
            </li>
            <li class="nav-item">
              <form method="post" action="{% url 'logout' %}" style="display: inline;">
                {% csrf_token %}
                  <button type="submit" class="nav-link btn btn-link" style="padding: 0; border: none; background: none;">
                   Выйти
                  </button>
              </form>
            </li>
          {% else %}
            <li class="nav-item">
              <a class="nav-link{% if view_name == 'login' %} active {% endif %}" href="{% url 'login' %}">Войти</a>
            </li>
            <!-- Новая кнопка -->
            <li class="nav-item">
              <a class="nav-link{% if view_name == 'registration' %} active {% endif %}" href="{% url 'registration' %}">Регистрация</a>
            </li>
          {% endif %}
        </ul>
      {% endwith %}
    </div>
  </nav>
</header> 
```